In [ ]:
import os
import sys
import time
import timeit
import traceback as tb
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution

from hmg import HBV001A

In [ ]:
# NSE -> Objective Function
#NSE
def nse(obs,sim):
   dnm = np.sum((obs - obs.mean())**2) 
   return  1.0 - np.sum((obs - sim)**2) / dnm

# Get the inputs
# Absolute path to the directory where the input data lies.
main_dir = Path(r'') # Here you paste your path
os.chdir(main_dir)

# Read input text time series as a pandas Dataframe object and
# cast the index to a datetime object.
inp_dfe = pd.read_csv(r'time_series___24163005.csv', sep=';', index_col=0)
inp_dfe.index = pd.to_datetime(inp_dfe.index, format='%Y-%m-%d-%H')

# Read the catcment area in meters squared. The first value is needed
# only.
cca_srs = pd.read_csv(r'area___24163005.csv', sep=';', index_col=0)
ccaa = cca_srs.values[0, 0]

tems = inp_dfe.loc[:, 'tavg__ref'].values  # Temperature.
ppts = inp_dfe.loc[:, 'pptn__ref'].values  # Preciptiation.
pets = inp_dfe.loc[:, 'petn__ref'].values  # PET.
diso = inp_dfe.loc[:, 'diso__ref'].values  # Observed discharge.

tsps = tems.shape[0]  # Number of time steps.

# Conversion constant for mm/hour to m3/s.
dslr = ccaa / (3600 * 1000)  # For daily res. multiply denominator with 24.

 # Initiate an empty model object. To understand what each method call
    # used below is for, please take a look at the files inside models
    # directory.
modl_objt = HBV001A()

# Set the above defined inputs.
modl_objt.set_inputs(tems, ppts, pets)

# Pass the number of time steps to the model object here. It creates the
# ouputs array(s) with the proper shape.
modl_objt.set_outputs(tsps)

# Set the constant that will convert units from those of precipitation
# to those of measured discharge.
modl_objt.set_discharge_scaler(dslr)

# Get a dictionary that links an output labe to its column index in the
# ouputs array.
otps_lbls = modl_objt.get_output_labels()


# Tell the model object that the simulation is a not an optimization.
modl_objt.set_optimization_flag(1)

def ObjectiveFunctionValue(parameters, modl_objt, diso):
    # Pass the parameters.
    modl_objt.set_parameters(parameters)

    # Run the model for the given inputs, constants and parameters.
    modl_objt.run_model()
    
    diss_simulated = modl_objt.get_discharge() # we want to optimize for this

    OFV = 1-nse(diso, diss_simulated)
    return OFV


# # Read the internal ouputs and simulated discharge.
    otps = modl_objt.get_outputs() # Need it in the final run

# MISSING: A CONTAINER that safes the best parameters and OFV !!!!!
# We have to pass model etc. 
def Optimize(bounds,objective_function):
    result = differential_evolution(objective_function, bounds) # There is a flag called polish, set it to false!!!
    return result.x #we have to rerun the model with these parameters to get the outputs

optimal_para = optim(...)

# Log NSE
obsl = np.log1p(obs) 
siml = np.log1p(sim) 
dnml = np.sum((obsl - obsl.mean())**2) 
lognse = 1 - np.sum((obsl - siml)**2) / dnml

#PBias
pbias = 100.0 * (np.sum(sim - obs) / np.sum(obs)) if np.sum(obs) != 0 else np.nan